In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import optuna
import random
from tqdm.notebook import tqdm

from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.feature_selection import RFECV, SelectKBest, f_classif
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from concurrent.futures import ThreadPoolExecutor

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from pprint import pprint
import os

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

experiment_name = 'multi-models_with_original_dataset'

In [2]:
train = pd.read_csv('train.csv')
original = pd.read_csv('Fault.csv', delimiter='\t')
test = pd.read_csv('test.csv')

In [3]:
train.shape, original.shape, test.shape

((19219, 35), (1940, 34), (12814, 28))

In [4]:
# Remove id column and check the dataset
train.drop('id', axis=1, inplace=True)
train.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [5]:
test.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,19219,1015,1033,3826564,3826588,659,23,46,62357,67,127,1656,0,1,150,0.3877,0.4896,0.3273,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,19220,1257,1271,419960,419973,370,26,28,39293,92,132,1354,0,1,40,0.1629,0.4136,0.0938,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,19221,1358,1372,117715,117724,289,36,32,29386,101,134,1360,0,1,40,0.0609,0.6234,0.4762,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,19222,158,168,232415,232440,80,10,11,8586,107,140,1690,1,0,100,0.4439,0.3333,0.8182,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,19223,559,592,544375,544389,140,19,15,15524,103,134,1688,1,0,60,0.8191,0.2619,0.4286,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170


In [6]:
original_dataset_cols = list(train.columns)

In [7]:
features_list = ['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
                 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
                 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
                 'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
                 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
                 'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
                 'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
                 'SigmoidOfAreas']

In [8]:
original.columns = original_dataset_cols
original.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
1,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
2,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
3,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
4,430,441,100250,100337,630,20,87,62357,64,127,1387,0,1,40,0.6200,0.3417,0.1264,0.0079,0.5500,1.0000,1.0,2.7993,1.0414,1.9395,0.8736,-0.2267,0.9874,1,0,0,0,0,0,0


In [9]:
# Concat the train and original dataset
combined_df = pd.concat([train, original], axis=0).reset_index(drop=True)

In [10]:
# Got function from https://www.kaggle.com/code/thomasmeiner/ps4e3-eda-feature-engineering-model

def reformat_data(df: pd.DataFrame) -> pd.DataFrame:
    target_cols = [
        "Pastry", #4
        "Z_Scratch", #6
        "K_Scatch", #2
        "Stains", #5
        "Dirtiness", #1
        "Bumps", #0
        "Other_Faults", #3
    ]
    non_target_cols = df.drop(target_cols, axis=1).columns.to_list()
    
    binary_dfs = []
    
    for col in target_cols:
        temp_df = df.loc[:, non_target_cols + [col]]
        temp_df = temp_df.loc[temp_df[col] == 1].copy() # keep positives only
        temp_df[col] = col # target value is class name now
        temp_df = temp_df.rename(columns={col: "target"}) # make target col name uniform for final concat
        binary_dfs.append(temp_df)
        
    reformatted_df = pd.concat(binary_dfs)
    return reformatted_df

In [11]:
# def reformat_data(df: pd.DataFrame) -> pd.DataFrame:
#     target_cols = [
#         "Pastry",
#         "Z_Scratch",
#         "K_Scatch",
#         "Stains",
#         "Dirtiness",
#         "Bumps",
#         "Other_Faults",
#     ]
#     non_target_cols = df.drop(target_cols, axis=1).columns.to_list()

#     binary_dfs = []

#     for col in target_cols:
#         temp_df = df.loc[:, non_target_cols + [col]]
#         temp_df = temp_df.loc[temp_df[col] == 1].copy() # sub sample to lowest class
#         temp_df[col] = col # target value is class name now
#         temp_df = temp_df.rename(columns={col: "target"}) # make target col name uniform for final concat
#         binary_dfs.append(temp_df)

#     # collect non_defect rows
#     temp_df = df.loc[
#         (df["Pastry"] == 0) &
#         (df["Z_Scratch"] == 0) &
#         (df["K_Scatch"] == 0) &
#         (df["Stains"] == 0) &
#         (df["Dirtiness"] == 0) &
#         (df["Bumps"] == 0) &
#         (df["Other_Faults"] == 0)
#     ]
#     temp_df = temp_df.loc[: , non_target_cols]
#     temp_df["target"] = "No defect"
#     binary_dfs.append(temp_df)

#     reformatted_df = pd.concat(binary_dfs)
#     return reformatted_df

In [12]:
combined_df = reformat_data(combined_df)
combined_df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,target
7,1673,1687,294065,294091,571,38,57,53142,77,110,1692,0,1,300,0.1491,0.4326,0.9643,0.0142,0.5686,0.7179,1.0,2.7528,1.3802,1.7559,0.0357,-0.2661,0.9408,Pastry
24,1538,1549,849219,849235,275,19,32,28986,71,117,1626,1,0,70,0.1494,0.3990,0.3750,0.0088,0.6316,1.0000,1.0,2.4393,1.0792,1.5051,0.6250,-0.2988,0.6330,Pastry
35,1310,1316,435871,435916,153,16,32,17101,104,132,1352,0,1,40,0.0532,0.3854,0.3333,0.0044,0.3750,0.9688,1.0,2.1847,0.7782,1.5051,0.6667,-0.0916,0.4025,Pastry
60,765,774,6571361,6571375,59,9,12,6682,77,133,1360,0,1,100,0.3613,0.3571,0.2800,0.0052,0.7778,1.0000,1.0,1.7708,0.8451,1.0792,0.7200,-0.1522,0.1892,Pastry
67,1677,1686,1319063,1319076,91,10,15,5608,57,95,1692,1,0,70,0.0024,0.3583,0.6667,0.0047,0.8000,1.0000,1.0,1.9590,0.9031,1.1761,0.3333,-0.3868,0.2660,Pastry


In [13]:
TARGET = 'target'

In [14]:
label_encoder = LabelEncoder()

label_encoder.fit(combined_df[TARGET])

combined_df[TARGET] = label_encoder.transform(combined_df[TARGET])

combined_df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,target
7,1673,1687,294065,294091,571,38,57,53142,77,110,1692,0,1,300,0.1491,0.4326,0.9643,0.0142,0.5686,0.7179,1.0,2.7528,1.3802,1.7559,0.0357,-0.2661,0.9408,4
24,1538,1549,849219,849235,275,19,32,28986,71,117,1626,1,0,70,0.1494,0.3990,0.3750,0.0088,0.6316,1.0000,1.0,2.4393,1.0792,1.5051,0.6250,-0.2988,0.6330,4
35,1310,1316,435871,435916,153,16,32,17101,104,132,1352,0,1,40,0.0532,0.3854,0.3333,0.0044,0.3750,0.9688,1.0,2.1847,0.7782,1.5051,0.6667,-0.0916,0.4025,4
60,765,774,6571361,6571375,59,9,12,6682,77,133,1360,0,1,100,0.3613,0.3571,0.2800,0.0052,0.7778,1.0000,1.0,1.7708,0.8451,1.0792,0.7200,-0.1522,0.1892,4
67,1677,1686,1319063,1319076,91,10,15,5608,57,95,1692,1,0,70,0.0024,0.3583,0.6667,0.0047,0.8000,1.0000,1.0,1.9590,0.9031,1.1761,0.3333,-0.3868,0.2660,4


In [15]:
combined_df.shape, test.shape

((20362, 28), (12814, 28))

In [16]:
X = combined_df.drop([TARGET], axis=1)
y = combined_df[TARGET]

n_splits = 3
sk10 = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=5)

In [24]:
models = [
    LGBMClassifier(n_jobs=-1, random_state=5),
    XGBClassifier(random_state=5),
    RandomForestClassifier(random_state=5),
    ExtraTreesClassifier(random_state=5),
    HistGradientBoostingClassifier(random_state=5),
    CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100),
    ]

In [18]:
def evaluate_models_roc(models, X, y, important_features, cv_split, experiment_name):
    MLA_compare = pd.DataFrame(columns=['MLA Name', 
                                        'MLA Parameters', 
                                        'MLA Train ROC AUC', 
                                        'MLA Test ROC AUC', 
                                        'MLA Test ROC AUC Std', 
                                        'MLA Time'])
    
    def evaluate_model(alg, idx):
        MLA_name = alg.__class__.__name__
        features = important_features.get(MLA_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {MLA_name} due to no important features.')
            return {
                'MLA Name': MLA_name,
                'MLA Parameters': str(alg.get_params()),
                'MLA Train ROC': 0,
                'MLA Test ROC': 0,
                'MLA Test ROC Std': 0,
                'MLA Time': "0 min 0.00 sec",
            }
        
        cv_results = cross_validate(alg, 
                                    X[features], 
                                    y, cv=cv_split, 
                                    scoring='roc_auc_ovr', 
                                    return_train_score=True, 
                                    n_jobs=-1)

        # Time formatting
        mean_fit_time = cv_results['fit_time'].mean()
        minutes, seconds = divmod(mean_fit_time, 60)

        # Results population
        result = {
            'MLA Name': MLA_name,
            'MLA Parameters': str(alg.get_params()),
            'MLA Train ROC AUC': cv_results['train_score'].mean(),
            'MLA Test ROC AUC': cv_results['test_score'].mean(),
            'MLA Test ROC AUC Std': cv_results['test_score'].std(),
            'MLA Time': f"{int(minutes)} min {seconds:.2f} sec",
        }

        print(f'Done with {MLA_name}.')
        return result

    results_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(models)]
        for future in futures:
            result = future.result()
            results_list.append(result)

    MLA_compare = pd.DataFrame(results_list)

    MLA_compare.sort_values(by=['MLA Test ROC AUC'], ascending=False, inplace=True)
    MLA_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return MLA_compare

In [26]:
baseline_features = {}

for model in models:
    model_name = model.__class__.__name__

    baseline_features[model_name] = list(X.columns)

In [20]:
%%time

baseline_models = evaluate_models_roc(models, X, y, baseline_features, sk10, f'{experiment_name}_lgbm')
baseline_models

Done with LGBMClassifier.
CPU times: total: 78.1 ms
Wall time: 8.93 s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.990784,0.892489,0.001999,0 min 3.98 sec


In [27]:
%%time

baseline_models = evaluate_models_roc(models, X, y, baseline_features, sk10, f'{experiment_name}')
baseline_models

Done with ExtraTreesClassifier.
Done with RandomForestClassifier.
Done with LGBMClassifier.
Done with HistGradientBoostingClassifier.
Done with XGBClassifier.
Done with CatBoostClassifier.
CPU times: total: 172 ms
Wall time: 1min 53s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
5,CatBoostClassifier,"{'verbose': False, 'random_state': 5, 'early_s...",0.986070,0.894479,0.002914,1 min 35.17 sec
4,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.967729,0.892836,0.002724,0 min 7.33 sec
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.990784,0.892489,0.001999,0 min 5.04 sec
1,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...",0.997369,0.888826,0.001681,0 min 55.27 sec
2,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.999999,0.887245,0.002894,0 min 10.40 sec
3,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",1.000000,0.881987,0.003742,0 min 4.05 sec


- Remove Correlated Features

In [21]:
# Remove correlated features (leaving just 1 of each pair)
# Leave features highly correlated with the target
df_no_corr = X.copy()
correlation_matrix_spear = df_no_corr.corr(method='spearman').abs()

# Select upper triangle of correlation matrix
upper_spear = correlation_matrix_spear.where(np.triu(np.ones(correlation_matrix_spear.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than a threshold (e.g., 0.9 in this case)
to_drop_spear = [column for column in upper_spear.columns if any(upper_spear[column] >= 0.9)]

# Drop features
df_reduced_spear = df_no_corr.drop(to_drop_spear, axis=1)

# Get list of low correlation features excluding TARGET
low_corr_feats_spear = list(df_reduced_spear.columns)

with open('low_corr_spear.txt', 'w') as f:
    f.write(str(low_corr_feats_spear))
    f.write('\n')

# Print the high correlation features effect
# Both pre and post drop dfs contain the TARGET
print(f"Dropped {len(to_drop_spear)} highly correlated features.\nOld Shape of the dataset was {df_no_corr.shape}\nNew shape of the dataset is {df_reduced_spear.shape}")

Dropped 9 highly correlated features.
Old Shape of the dataset was (20362, 27)
New shape of the dataset is (20362, 18)


In [28]:
%%time

no_corr_features = {}

for model in models:
    model_name = model.__class__.__name__

    no_corr_features[model_name] = list(df_reduced_spear.columns)

CPU times: total: 0 ns
Wall time: 0 ns


In [29]:
%%time

no_corr_models = evaluate_models_roc(models, df_reduced_spear, y, no_corr_features, sk10, f'{experiment_name}_corr')
no_corr_models

Done with LGBMClassifier.
Done with ExtraTreesClassifier.
Done with RandomForestClassifier.
Done with HistGradientBoostingClassifier.
Done with XGBClassifier.
Done with CatBoostClassifier.
CPU times: total: 78.1 ms
Wall time: 1min 12s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
5,CatBoostClassifier,"{'verbose': False, 'random_state': 5, 'early_s...",0.986033,0.893920,0.002013,1 min 4.51 sec
4,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.966187,0.893112,0.002489,0 min 4.99 sec
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.989215,0.892714,0.002259,0 min 3.13 sec
1,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...",0.996809,0.889151,0.001552,0 min 32.58 sec
2,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.999999,0.887550,0.001453,0 min 6.56 sec
3,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",1.000000,0.883300,0.002424,0 min 3.80 sec


- Feature Importances

In [ ]:
# feat_importance_features = {}

# for model in models:
#     model_name = model.__class__.__name__

#     try:
#         # Initialize array to store feature importances
#         feature_importances = np.zeros(X.shape[1])

#         # Loop through each fold and calculate the feature importances
#         for train_index, test_index in sk10.split(X, y):
#             X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#             y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#             model.fit(X_train, y_train)

#             # Get the feature importances and them to the total
#             feature_importances += model.feature_importances_

#         feature_importances /= n_splits

#         feature_importances_dict = dict(zip(X.columns, feature_importances))

#         df = pd.DataFrame.from_dict(feature_importances_dict, orient='index')

#         # Resetting index with a name for the column
#         df = df.reset_index().rename(columns={'index': 'Feature', 0: 'Avg_Feat_Importance'})
#         df.sort_values(by='Avg_Feat_Importance', ascending=False, inplace=True)

#         # Save to CSV
#         df.to_csv(f'{model_name}_feature_importances.csv')

#         fi_threshold = 0

#         fi_feats = df[df['Avg_Feat_Importance'] > fi_threshold]['Feature'].tolist()

#         feat_importance_features[model_name] = fi_feats
#         print(f'Done with {model_name}')

#     except AttributeError:
#         feat_importance_features[model_name] = list(X.columns)
#         print(f'{model_name} does not have feature_importances_')

In [ ]:
# with open('featimp_features.txt', mode='w') as f:
#     pprint(feat_importance_features, stream=f)

- Permutation Importance

In [30]:
# Generate a random feature for X
np.random.seed(5)
df_reduced_spear['random_control_feature'] = np.round(np.random.uniform(-2, 2, df_reduced_spear.shape[0]), 6)
df_reduced_spear.shape

(20362, 19)

In [31]:
%%time

perm_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

perm_importances = {model.__class__.__name__: [] for model in models}

for i, (train_idx, test_idx) in enumerate(perm_cv.split(df_reduced_spear, y)):
    X_train, X_test = df_reduced_spear.iloc[train_idx], df_reduced_spear.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y_train)
        # Calculate permutation importance
        result = permutation_importance(model, X_test, y_test, n_repeats=5, random_state=5, n_jobs=-1, scoring='roc_auc_ovr')
        perm_importances[model_name].append(result.importances_mean)
        print(f'Done with {model_name}.')
    
    print(f'Done with Fold {i+1}', end='\n\n')

Done with LGBMClassifier.
Done with XGBClassifier.
Done with RandomForestClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with CatBoostClassifier.
Done with Fold 1

Done with LGBMClassifier.
Done with XGBClassifier.
Done with RandomForestClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with CatBoostClassifier.
Done with Fold 2

Done with LGBMClassifier.
Done with XGBClassifier.
Done with RandomForestClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with CatBoostClassifier.
Done with Fold 3

Done with LGBMClassifier.
Done with XGBClassifier.
Done with RandomForestClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with CatBoostClassifier.
Done with Fold 4

Done with LGBMClassifier.
Done with XGBClassifier.
Done with RandomForestClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with Cat

In [32]:
%%time

# Average importances across folds and export to CSV
for model_name, importances in perm_importances.items():
    avg_importance = np.mean(importances, axis=0)
    importance_df = pd.DataFrame({'Feature': df_reduced_spear.columns, 'Importance': avg_importance})
    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    # Export to CSV
    importance_df.to_csv(f'.\permutation_importances\{model_name}_permutation_importance.csv', index=False)

print('Done with Permuation Importances', end='\n\n')

Done with Permuation Importances

CPU times: total: 0 ns
Wall time: 22.8 ms


In [33]:
directory = 'permutation_importances'

# Initialize a dictionary for the features
perm_important_features = {}

for model in models:
    model_name = model.__class__.__name__
    csv_path = os.path.join(directory, f'{model_name}_permutation_importance.csv')
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)

        # Check for 'random_control_feature' and its importance
        if 'random_control_feature' in df['Feature'].values:
            random_feature_importance = df.loc[df['Feature'] == 'random_control_feature', 'Importance'].iloc[0]
        else:
            random_feature_importance = 0

        # Determine the threshold
        threshold = max(0, random_feature_importance)

        # Filter features where importance is greater than 0
        important_feats_filtered = df[df['Importance'] > threshold]['Feature'].tolist()

        # Reorder important_feats based on the predefined features_list
        important_feats_ordered = [feat for feat in features_list if feat in important_feats_filtered]

        # Add to importance dictionary
        perm_important_features[model_name] = important_feats_ordered

    else:
        print(f'CSV file for {model_name} not found.')

print('Done getting important features dictionary')

Done getting important features dictionary


In [34]:
with open('perm_important_features_lgbm.txt', mode='w') as f:
    pprint(perm_important_features, stream=f)

In [35]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

perm_importance_models = evaluate_models_roc(models, X, y, perm_important_features, sk10, f'{experiment_name}_permimp')
perm_importance_models

Done with RandomForestClassifier.
Done with ExtraTreesClassifier.
Done with LGBMClassifier.
Done with HistGradientBoostingClassifier.
Done with XGBClassifier.
Done with CatBoostClassifier.
CPU times: total: 141 ms
Wall time: 1min 18s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
5,CatBoostClassifier,"{'verbose': False, 'random_state': 5, 'early_s...",0.986033,0.893920,0.002013,1 min 8.49 sec
4,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.966187,0.893112,0.002489,0 min 6.41 sec
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.989215,0.892714,0.002259,0 min 3.50 sec
1,XGBClassifier,"{'objective': 'multi:softprob', 'use_label_enc...",0.996809,0.889151,0.001552,0 min 33.76 sec
2,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.999999,0.887550,0.001453,0 min 6.50 sec
3,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",1.000000,0.883300,0.002424,0 min 3.76 sec


- SelectKBest with f_classif

In [ ]:
best_features_list = []
kbest_features = {}

for model in models:
    model_name = model.__class__.__name__

    # Select whichever one had a better CV score generally
    # Also, consider computational expense and accuracy balance
    
    features = perm_important_features[model_name]
    # features = list(df_reduced_spear.columns)

    # incase there is no feature that had importance, go to the next model
    if len(features) == 0:
        continue
	
    X_kbest = X[features]
    best_score = 0
    best_k = 0
    best_features = []

    # Iterate over k from 1 to number of features
    for k in range(1, len(features) + 1):
        print(f'currently running {k} features on {model_name}')
        # Apply SelectKBest
        selector = SelectKBest(f_classif, k=k)
        X_new = selector.fit_transform(X_kbest, y)

        # Get the selected feature names
        selected_features = X_kbest.columns[selector.get_support()]

        # Evaluate the model
        # model = LGBMClassifier(n_jobs=-1, random_state=5)
        roc_auc_scores = cross_validate(model, X_new, y, cv=sk10, scoring='roc_auc_ovr', n_jobs=-1)
        mean_roc_auc_scores = roc_auc_scores['test_score'].mean()

        if mean_roc_auc_scores > best_score:
            best_k = k
            best_score = mean_roc_auc_scores
            best_features = list(selected_features)

    best_features_list.append({'k': best_k,
                    'Selected Features': best_features,
                    'ROC AUC Score': best_score,
                    'Model Name': model_name})
    
    kbest_features[model_name] = best_features

best_features_df = pd.DataFrame(best_features_list)

best_features_df.sort_values(by='ROC AUC Score', ascending=False, inplace=True)

In [ ]:
with open('kbest_features.txt', mode='w') as f:
    pprint(kbest_features, stream=f)

In [ ]:
best_features_df

- RFECV

In [36]:
%%time

# Initialize empty dictionary for RFECV features
rfecv_features = {}

for alg in models:
    # set name
    MLA_name = alg.__class__.__name__
		
    features = perm_important_features[MLA_name]

    # incase there is no feature that had importance, go to the next model
    if len(features) == 0:
        continue
	
    X_rfecv = X[features]

    try:
        print(f'Starting with {MLA_name}')
        # Create the RFECV object and rank each feature
        selector = RFECV(alg, cv=sk10, step=1, scoring='roc_auc_ovr', verbose=2)
        selector = selector.fit(X_rfecv, y)

        selected_features = list(X_rfecv.columns[selector.support_])

        # Reorder selected_features based on the predefined features_list
        selected_features_ordered = [feat for feat in features_list if feat in selected_features]

        rfecv_features[MLA_name] = selected_features_ordered

        print(f'Done with {MLA_name}', end='\n\n')
    
    except ValueError:
        # In case of an error, keep the original order but filtered by features_list
        features_filtered = [feat for feat in features_list if feat in features]
        rfecv_features[MLA_name] = features_filtered
        print(f'{MLA_name} does not have coef_ or feature_importances_', end='\n\n')

Starting with LGBMClassifier
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting 

In [37]:
with open('rfecv_features.txt', mode='w') as f:
    pprint(rfecv_features, stream=f)

In [38]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

rfecv_models = evaluate_models_roc(models, X, y, rfecv_features, sk10, f'{experiment_name}_rfecv')
rfecv_models

Done with HistGradientBoostingClassifier.
Done with LGBMClassifier.
Done with RandomForestClassifier.
Done with XGBClassifier.
Done with ExtraTreesClassifier.
Done with CatBoostClassifier.
CPU times: total: 156 ms
Wall time: 1min 21s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
5,CatBoostClassifier,"{'verbose': False, 'random_state': 5, 'early_s...",0.986033,0.893920,0.002013,0 min 59.70 sec
4,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.966187,0.893112,0.002489,0 min 5.06 sec
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.989215,0.892714,0.002259,0 min 5.10 sec
1,XGBClassifier,"{'objective': 'multi:softprob', 'use_label_enc...",0.995133,0.890407,0.001849,0 min 32.45 sec
2,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.999999,0.888379,0.003159,0 min 10.75 sec
3,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",1.000000,0.883300,0.002424,0 min 5.45 sec


- SFS

In [39]:
%%time

# Initialize empty dictionary for SFS features
sfs_features = {}

for alg in models:
    # set name
    MLA_name = alg.__class__.__name__

    try:
            
        # features = kbest_features[MLA_name]
        # features = feat_importance_features[MLA_name]
        features = rfecv_features[MLA_name]

        # incase there is no feature that had importance, go to the next model
        if len(features) == 0:
            continue
        
        X_sfs = X[features]

        print(f'Running backward feature selection with {MLA_name}')

        sfs = SFS(alg,
            k_features='best',
            forward=False,
            floating=False,
            scoring='roc_auc_ovr',
            verbose=2,
            n_jobs=-1,
            cv=sk10)
        
        sfs = sfs.fit(X_sfs, y)

        # Get the selected features index
        selected_sfs_idx = list(sfs.k_feature_idx_)

        # Get the feature names
        selected_sfs_feats = X_sfs.columns[selected_sfs_idx]

        selected_features = list(selected_sfs_feats)

        # Reorder selected_features based on the predefined features_list
        selected_features_ordered = [feat for feat in features_list if feat in selected_features]

        sfs_features[MLA_name] = selected_features_ordered

        print(f'Done with {MLA_name}', end='\n\n')

    except KeyError:
        print(f'{MLA_name} not in the dictionary.')

Running backward feature selection with LGBMClassifier


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:   35.2s remaining:   13.5s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   43.2s finished

[2024-03-26 14:43:41] Features: 17/1 -- score: 0.8931889102669391[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  17 | elapsed:   32.7s remaining:   17.8s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   40.5s finished

[2024-03-26 14:44:22] Features: 16/1 -- score: 0.8936959020766403[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:   31.2s remaining:   18.7s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   33.1s finished

[2024-03-26 14:44:55] Features: 15/1 -- score: 0.8933859012158504[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15

Done with LGBMClassifier

Running backward feature selection with XGBClassifier


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:  2.8min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  2.8min finished

[2024-03-26 14:51:59] Features: 15/1 -- score: 0.8905056320021748[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  1.4min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.6min finished

[2024-03-26 14:54:34] Features: 14/1 -- score: 0.8902649491866047[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  14 | elapsed:  1.3min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:  2.4min finished

[2024-03-26 14:57:00] Features: 13/1 -- score: 0.8896797088981815[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  13

Done with XGBClassifier

Running backward feature selection with RandomForestClassifier


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed:   40.9s remaining:   15.7s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   48.3s finished

[2024-03-26 15:27:47] Features: 17/1 -- score: 0.8939482997763749[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  17 | elapsed:   39.3s remaining:   21.4s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:   45.5s finished

[2024-03-26 15:28:33] Features: 16/1 -- score: 0.8941358697570916[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed:   36.9s remaining:   22.1s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:   40.5s finished

[2024-03-26 15:29:14] Features: 15/1 -- score: 0.8942280336666886[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15

Done with HistGradientBoostingClassifier

Running backward feature selection with CatBoostClassifier


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  18 | elapsed: 15.3min remaining:  5.9min
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 17.4min finished

[2024-03-26 15:52:16] Features: 17/1 -- score: 0.8947935801827138[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  17 | elapsed: 14.7min remaining:  8.0min
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed: 16.0min finished

[2024-03-26 16:08:17] Features: 16/1 -- score: 0.8947465481550395[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  16 | elapsed: 13.7min remaining:  8.2min
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed: 13.9min finished

[2024-03-26 16:22:11] Features: 15/1 -- score: 0.89551028294688[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 |

Done with CatBoostClassifier

CPU times: total: 9min 36s
Wall time: 3h 1min 25s


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   40.2s finished

[2024-03-26 17:44:19] Features: 1/1 -- score: 0.7718403465881392

In [42]:
with open('sfs_features.txt', mode='w') as f:
    pprint(sfs_features, stream=f)

In [41]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

sfs_models = evaluate_models_roc(models, X, y, sfs_features, sk10, f'{experiment_name}_sfs')
sfs_models

Done with RandomForestClassifier.
Done with LGBMClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with XGBClassifier.
Done with CatBoostClassifier.
CPU times: total: 156 ms
Wall time: 1min 5s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
5,CatBoostClassifier,"{'verbose': False, 'random_state': 5, 'early_s...",0.981920,0.895510,0.002137,0 min 52.77 sec
4,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.964685,0.894672,0.002302,0 min 4.09 sec
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.988834,0.893696,0.002014,0 min 3.90 sec
1,XGBClassifier,"{'objective': 'multi:softprob', 'use_label_enc...",0.994761,0.890506,0.001488,0 min 26.63 sec
2,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.999999,0.889755,0.002782,0 min 7.11 sec
3,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",1.000000,0.885806,0.002558,0 min 4.20 sec


In [ ]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

feat_importance_models = evaluate_models_roc(models, X, y, feat_importance_features, sk10, f'{experiment_name}_featimp')
feat_importance_models

In [ ]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

kbest_models = evaluate_models_roc(models, X, y, kbest_features, sk10, f'{experiment_name}_kbest')
kbest_models

# Ensembling

In [43]:
model1 = LGBMClassifier(n_jobs=-1, random_state=5)
model2 = XGBClassifier(random_state=5)
model3 = RandomForestClassifier(random_state=5)
model4 = ExtraTreesClassifier(random_state=5)
model5 = HistGradientBoostingClassifier(random_state=5)
model6 = CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100)

- Features for Competition + Original dataset down to SFS for all models (Experiment Set 2)

In [44]:
model1_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index']
model2_feats = ['X_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'Orientation_Index', 'Luminosity_Index']
model3_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index']
model4_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index']
model5_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'Minimum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index']
model6_feats = ['X_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'Orientation_Index', 'Luminosity_Index']

In [46]:
%%time

model1_results, model2_results, model3_results, model4_results, model5_results, model6_results, y_test_list = [], [], [], [], [], [], []

X_lgbm = X[model1_feats]
X_xgb = X[model2_feats]
X_rf = X[model3_feats]
X_extrat = X[model4_feats]
X_hist = X[model5_feats]
X_cat = X[model6_feats]

for i, (train_index, test_index) in enumerate(sk10.split(X, y)):
    X_train_lgbm, X_test_lgbm = X_lgbm.iloc[train_index], X_lgbm.iloc[test_index]
    X_train_xgb, X_test_xgb = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    X_train_rf, X_test_rf = X_rf.iloc[train_index], X_rf.iloc[test_index]
    X_train_extrat, X_test_extrat = X_extrat.iloc[train_index], X_extrat.iloc[test_index]
    X_train_hist, X_test_hist = X_hist.iloc[train_index], X_hist.iloc[test_index]
    X_train_cat, X_test_cat = X_cat.iloc[train_index], X_cat.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model1.fit(X_train_lgbm, y_train)
    model1_results.append(model1.predict_proba(X_test_lgbm))

    model2.fit(X_train_xgb, y_train)
    model2_results.append(model2.predict_proba(X_test_xgb))

    model3.fit(X_train_rf, y_train)
    model3_results.append(model3.predict_proba(X_test_rf))

    model4.fit(X_train_extrat, y_train)
    model4_results.append(model4.predict_proba(X_test_extrat))

    model5.fit(X_train_hist, y_train)
    model5_results.append(model5.predict_proba(X_test_hist))

    model6.fit(X_train_cat, y_train)
    model6_results.append(model6.predict_proba(X_test_cat))

    y_test_list.append(y_test)

    print(f'Done with fold {i+1}.')

Done with fold 1.
Done with fold 2.
Done with fold 3.
CPU times: total: 3min 48s
Wall time: 2min 7s


In [48]:
%%time

model1_weights, model2_weights, model3_weights, model4_weights, model5_weights, model6_weights, scores = [], [], [], [], [], [], []

for i in tqdm(range(20000)):
    weight_1 = np.random.random_sample(size=1)[0]
    weight_2 = np.random.random_sample(size=1)[0]
    weight_3 = np.random.random_sample(size=1)[0]
    weight_4 = np.random.random_sample(size=1)[0]
    weight_5 = np.random.random_sample(size=1)[0]
    weight_6 = np.random.random_sample(size=1)[0]

    model1_weights.append(weight_1)
    model2_weights.append(weight_2)
    model3_weights.append(weight_3)
    model4_weights.append(weight_4)
    model5_weights.append(weight_5)
    model6_weights.append(weight_6)

    scores_in = []

    for j in range(n_splits):
        weighted_pred = weight_1 * model1_results[j] + weight_2 * model2_results[j] + weight_3 * model3_results[j] + weight_4 * model4_results[j] + weight_5 * model5_results[j] + weight_6 * model6_results[j]
        weighted_pred_normalized = weighted_pred / np.sum(weighted_pred, axis=1, keepdims=True)
        scores_in.append(roc_auc_score(y_test_list[j], weighted_pred_normalized, multi_class='ovr'))
        
    scores.append(np.mean(scores_in))

  0%|          | 0/20000 [00:00<?, ?it/s]

CPU times: total: 7min 41s
Wall time: 18min 28s


In [49]:
results_df = pd.DataFrame()
results_df['model_1'] = model1_weights
results_df['model_2'] = model2_weights
results_df['model_3'] = model3_weights
results_df['model_4'] = model4_weights
results_df['model_5'] = model5_weights
results_df['model_6'] = model6_weights
results_df['score'] = scores
results_df = results_df.sort_values(by='score', ascending=False).reset_index(drop=True)
results_df.head(10)

,model_1,model_2,model_3,model_4,model_5,model_6,score
0,0.452483,0.003644,0.393666,0.250090,0.871584,0.708088,0.900483
1,0.295824,0.001112,0.271955,0.217891,0.661835,0.633712,0.900475
2,0.598383,0.042975,0.384315,0.390022,0.989833,0.811929,0.900472
3,0.340604,0.071712,0.409629,0.311217,0.677121,0.605779,0.900472
4,0.450208,0.001133,0.431225,0.363697,0.914132,0.598980,0.900469
5,0.539158,0.171186,0.405682,0.435197,0.982618,0.849286,0.900467
6,0.321890,0.191247,0.390652,0.241832,0.834249,0.697683,0.900465
7,0.496515,0.123318,0.400970,0.350710,0.772840,0.803171,0.900454
8,0.510782,0.231132,0.402910,0.373183,0.987360,0.853566,0.900454
9,0.474728,0.110021,0.559841,0.246916,0.977145,0.796581,0.900454


# Get Submission (Random Weight Ensemble)

In [50]:
%%time

model1_final = model1.fit(X_lgbm, y)
model2_final = model2.fit(X_xgb, y)
model3_final = model3.fit(X_rf, y)
model4_final = model4.fit(X_extrat, y)
model5_final = model5.fit(X_hist, y)
model6_final = model6.fit(X_cat, y)

CPU times: total: 1min 41s
Wall time: 55.3 s


In [51]:
ensemble_pred = (
                results_df['model_1'][0] * model1_final.predict_proba(test[model1_feats]) +
                results_df['model_2'][0] * model2_final.predict_proba(test[model2_feats]) +
                results_df['model_3'][0] * model3_final.predict_proba(test[model3_feats]) +
                results_df['model_4'][0] * model4_final.predict_proba(test[model4_feats]) +
                results_df['model_5'][0] * model5_final.predict_proba(test[model5_feats]) +
                results_df['model_6'][0] * model6_final.predict_proba(test[model6_feats])
                 )

ensemble_df = pd.DataFrame(ensemble_pred)

# If all models predict 0, instead of getting NaN, fill in 0
ensemble_df = ensemble_df.div(ensemble_df.sum(axis=1), axis=0).fillna(0)
ensemble_df.columns = label_encoder.classes_

In [52]:
ensemble_df.head()

,Bumps,Dirtiness,K_Scatch,Other_Faults,Pastry,Stains,Z_Scratch
0,0.113662,0.022089,0.003626,0.304330,0.548311,0.000055,0.007928
1,0.119760,0.216375,0.008641,0.372832,0.276360,0.000082,0.005950
2,0.285218,0.005803,0.067929,0.596786,0.002943,0.003305,0.038016
3,0.380233,0.008759,0.000421,0.468349,0.138443,0.001753,0.002041
4,0.689274,0.006771,0.000446,0.290896,0.004025,0.004275,0.004312


In [53]:
submission = pd.read_csv('sample_submission.csv')
submission_df = pd.concat([submission['id'], ensemble_df], axis=1)
submission_df.head()

,id,Bumps,Dirtiness,K_Scatch,Other_Faults,Pastry,Stains,Z_Scratch
0,19219,0.113662,0.022089,0.003626,0.304330,0.548311,0.000055,0.007928
1,19220,0.119760,0.216375,0.008641,0.372832,0.276360,0.000082,0.005950
2,19221,0.285218,0.005803,0.067929,0.596786,0.002943,0.003305,0.038016
3,19222,0.380233,0.008759,0.000421,0.468349,0.138443,0.001753,0.002041
4,19223,0.689274,0.006771,0.000446,0.290896,0.004025,0.004275,0.004312


In [54]:
submission_df.to_csv('submission_ensemble_3fold_0.900483.csv', index=False)